# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [ ]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [3]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 '

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [4]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [10]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
https://edwar

In [9]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [11]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'company page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'linkedin', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'twitter', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'facebook', 'url': 'https://www.facebook.com/edward.donner.52'}]}

In [13]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [14]:
select_relevant_links("https://edwarddonner.com")

Selecting relevant links for https://edwarddonner.com by calling gpt-5-nano
Found 2 relevant links


{'links': [{'type': 'company page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'}]}

In [15]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 11 relevant links


{'links': [{'type': 'company homepage', 'url': 'https://huggingface.co/'},
  {'type': 'brand page', 'url': 'https://huggingface.co/brand'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'community page', 'url': 'https://huggingface.co/join/discord'},
  {'type': 'GitHub', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'discuss forum', 'url': 'https://discuss.huggingface.co'},
  {'type': 'endpoints product page',
   'url': 'https://endpoints.huggingface.co'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [16]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [17]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 9 relevant links
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
MiniMaxAI/MiniMax-M2
Updated
5 days ago
•
726k
•
968
deepseek-ai/DeepSeek-OCR
Updated
9 days ago
•
2.06M
•
2.39k
moonshotai/Kimi-Linear-48B-A3B-Instruct
Updated
2 days ago
•
15k
•
314
briaai/FIBO
Updated
about 20 hours ago
•
2.84k
•
188
dx8152/Qwen-Edit-2509-Multiple-angles
Updated
about 4 hours ago
•
144
Browse 1M+ models
Spaces
Running
on
CPU Upgrade
1.09k
1.09k
The Smol Training Playbook: The Secrets to Building World-Class LLMs
📝
Running
15.6k
15.6k
DeepSite v3
🐳
Generate any application by Vibe Coding
Running
2.24k
2.24k
Wan2.2 A

In [18]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """


In [20]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [21]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 10 relevant links


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nMiniMaxAI/MiniMax-M2\nUpdated\n5 days ago\n•\n726k\n•\n968\ndeepseek-ai/DeepSeek-OCR\nUpdated\n9 days ago\n•\n2.06M\n•\n2.39k\nmoonshotai/Kimi-Linear-48B-A3B-Instruct\nUpdated\n2 days ago\n•\n15k\n•\n314\nbriaai/FIBO\nUpdated\nabout 20 hours ago\n•\n2.84k\n•\n188\ndx8152/Qwen-Edit-2509-Multiple-angles\nUpdated\nabout 5 hours ago\n•\n145\nBrowse 1M+ models\nSpaces\nRunning\non\nCPU Upgrade\n

In [22]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [23]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 10 relevant links


# Hugging Face Brochure

---

## About Hugging Face

**Hugging Face** is the premier AI community and collaborative platform dedicated to building the future of machine learning (ML). It is where ML practitioners, researchers, and developers come together to share, discover, and collaborate on models, datasets, and AI applications. As a central hub for the machine learning community, Hugging Face promotes an open and ethical AI future.

---

## What We Offer

- **Collaborative Platform:**  
  Host and collaborate on unlimited public models, datasets, and applications. Enables sharing and discovery across millions of assets.

- **Models & Datasets:**  
  Access over 1 million open-source machine learning models and more than 250,000 datasets — spanning text, image, video, audio, and even 3D modalities.

- **Spaces:**  
  Run and explore AI applications hosted directly on the platform, offering interactive demos and live ML-powered apps.

- **Compute and Enterprise Solutions:**  
  Accelerate your ML projects with paid compute resources and enterprise-grade tools built for teams and organizations seeking advanced AI capabilities.

---

## Community & Culture

Hugging Face is much more than a platform—it’s a vibrant global community of machine learning engineers, scientists, and end-users committed to sharing knowledge and pushing the boundaries of AI. The company fosters a culture of:

- **Open Collaboration:** Encouraging transparent and ethical sharing of machine learning work.  
- **Innovation:** Empowering creators and researchers with easy access to cutting-edge tools and resources.  
- **Inclusivity:** Building an accessible space where diverse voices in AI can contribute and grow.

---

## Key Features & Highlights

- **The Hugging Face Hub:**  
   A central repository to explore, experiment, and deploy open-source ML models and datasets.

- **Explore AI Apps:**  
   Browse 400,000+ applications running live on Hugging Face’s Spaces infrastructure.

- **Multi-Modality Support:**  
   Work with models across various data types including text, images, video, audio, and 3D environments.

- **Build Your ML Portfolio:**  
   Share your work with the global machine learning community and build your professional profile.

---

## Our Customers

Hugging Face serves a broad spectrum of customers including:

- Individual ML practitioners and researchers  
- Academic and scientific communities  
- AI startups and technology companies  
- Large enterprises seeking to accelerate AI adoption with scalable solutions

Notable collaborations include partnerships and dataset contributions from industry leaders like NVIDIA.

---

## Careers at Hugging Face

Join a fast-growing, innovative tech company on a mission to democratize AI. Hugging Face offers roles for engineers, researchers, product managers, and community builders who are passionate about machine learning and open-source AI. Working here means being part of a highly collaborative, purpose-driven culture that values creativity, transparency, and impact.

---

## Brand Identity

- **Logo & Colors:**  
   The brand is recognized by its vibrant yellow (#FFD21E, #FF9D00) and modern grey (#6B7280) palette, symbolizing creativity and professionalism.

- **Tagline:**   
   “The AI community building the future.”

---

## Get Started with Hugging Face

Whether you want to explore 1M+ ML models, host your datasets, or launch AI applications, Hugging Face provides the tools and community support to advance your ML journey.  
Sign up today and be a part of the future of AI collaboration.

---

**Website:** [huggingface.co](https://huggingface.co)  
**Explore Models:** https://huggingface.co/models  
**Explore Datasets:** https://huggingface.co/datasets  
**Spaces (Apps):** https://huggingface.co/spaces  
**Careers:** Available on the website

---

*Hugging Face — Empowering open and ethical AI with community-driven innovation.*

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [24]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [25]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 18 relevant links


# Hugging Face Brochure

## About Hugging Face

Hugging Face is a pioneering AI company and the vibrant community at the forefront of machine learning innovation. The company offers a collaborative platform where ML engineers, scientists, and enthusiasts come together to create, share, and explore models, datasets, and applications. Hugging Face empowers the future of artificial intelligence with an open, ethical, and inclusive approach.

## What We Offer

- **Hugging Face Hub:** A central repository hosting over 1 million public machine learning models, 250k+ datasets, and 400k+ AI applications spanning multiple modalities such as text, image, video, audio, and even 3D.
- **Spaces:** Interactive environments where users can deploy and test models and apps with ease, supported by scalable compute options including ZeroGPU.
- **Open-Source Libraries & Tools:** Highly popular ML open-source libraries, enabling rapid AI innovation and experimentation.
- **Enterprise Solutions:** Advanced platform tailored for organizations, featuring enterprise-grade security, single sign-on (SSO), access control, audit logs, analytics, private storage, dedicated support, and flexible compute options.

## Community & Culture

- **Collaborative Spirit:** Hugging Face fosters a global AI community where learning, collaboration, and sharing knowledge drive progress.
- **Ethical AI:** Passionate about building AI that is responsible, open, and accessible to all.
- **Innovation-Driven:** The team includes talented scientists and engineers pushing the boundaries of ML technology.
- **Open & Transparent:** Encourages researchers and developers worldwide to contribute, experiment, and build AI portfolios visible to the community.

## Customers & Users

Hugging Face’s platform serves a broad spectrum of users including:

- **Machine Learning Engineers and Researchers:** Access to cutting-edge models and datasets accelerates experimentation.
- **Enterprises & Teams:** Scalable tools and secure infrastructure that support building AI products and workflows.
- **Educators & Students:** Resources to learn and build AI skills.
- **Developers:** Ready-to-use APIs and applications for immediate integration into products.

## Careers at Hugging Face

Join a fast-growing, innovative AI enterprise committed to shaping the future of machine learning. Careers at Hugging Face offer the chance to:

- Work with leading open-source AI projects and a global community.
- Collaborate with passionate experts dedicated to advancing AI technology.
- Contribute to ethical AI initiatives and an inclusive company culture.
- Grow your skills in a dynamic environment at the cutting edge of machine learning.

Explore current openings on the Hugging Face Careers page to find your place in the AI revolution.

## Brand & Identity

Hugging Face’s brand reflects its energetic and innovative culture, highlighted by distinctive colors (#FFD21E, #FF9D00, #6B7280) and a friendly logo representing connection and collaboration.

---

**Discover, Build, and Scale AI with Hugging Face — the AI community building the future.**

Visit: [huggingface.co](https://huggingface.co)  
Explore Models, Datasets, Spaces, Enterprise solutions & More!

---

**Contact Us**  
For Enterprise inquiries: sales@huggingface.co  
Join the community on [GitHub](https://github.com/huggingface), [Twitter](https://twitter.com/huggingface), [LinkedIn](https://linkedin.com/company/huggingface), and [Discord](https://discord.gg/huggingface)

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>